In [ ]:
import sys
sys.path.append('..')

from src.preprocessing import ImagePreprocessor
from src.detection import ChangeDetector
from src.evaluation import ChangeDetectionEvaluator
from src.visualization import ResultVisualizer

In [ ]:
# Загрузка конфигурации
with open('../config/params.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
# Загрузка тестовых данных
print("Загрузка тестовых данных...")
img1 = cv2.imread('../data/raw/test_T1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('../data/raw/test_T2.png', cv2.IMREAD_GRAYSCALE)
gt = cv2.imread('../data/raw/test_GT.png', cv2.IMREAD_GRAYSCALE)

print(f"Размеры: T1={img1.shape}, T2={img2.shape}, GT={gt.shape}")

In [ ]:
# Предобработка
print("\nПредобработка изображений...")
preprocessor = ImagePreprocessor(config['preprocessing'])
img1_proc, img2_proc = preprocessor.process_pair(img1, img2)

In [ ]:
# Эксперимент 1: Сравнение методов
print("\nЭксперимент 1: Сравнение методов детекции...")
methods = ['differencing', 'ratio', 'cva', 'ndbi_diff']
results = {}

for method in methods:
    print(f"  Тестирование метода: {method}")
    detector = ChangeDetector(method=method, params=config['detection'])
    mask, info = detector.detect(img1_proc, img2_proc)
    
    evaluator = ChangeDetectionEvaluator()
    metrics = evaluator.compute_metrics(mask, gt)
    
    results[method] = {
        'mask': mask,
        'info': info,
        'metrics': metrics
    }

In [ ]:
# Визуализация сравнения
print("\nВизуализация результатов...")
visualizer = ResultVisualizer()

for method, result in results.items():
    fig = visualizer.create_comparison_figure(
        img1_proc, img2_proc,
        result['mask'], result['mask'],
        ground_truth=gt,
        title=f"Метод: {method.upper()}"
    )
    
    plt.show()
    
    # Сохранение результатов
    visualizer.save_visualization(
        fig,
        f'../results/comparison_{method}.png'
    )

# Анализ влияния параметров
print("\nЭксперимент 2: Анализ влияния параметров...")

In [ ]:
# Исследование пороговых значений
thresholds = np.linspace(0.1, 0.5, 9)
f1_scores = []

for thresh in thresholds:
    detector = ChangeDetector(
        method='cva',
        params={'manual_threshold': thresh}
    )
    mask, _ = detector.detect(img1_proc, img2_proc)
    
    evaluator = ChangeDetectionEvaluator()
    metrics = evaluator.compute_metrics(mask, gt)
    f1_scores.append(metrics['f1_score'])

In [ ]:
# График зависимости F1 от порога
plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, 'o-', linewidth=2, markersize=8)
plt.xlabel('Пороговое значение')
plt.ylabel('F1-Score')
plt.title('Зависимость точности от порога (метод CVA)')
plt.grid(True, alpha=0.3)
plt.savefig('../results/threshold_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

# Сохранение итоговой таблицы
print("\nГенерация итогового отчета...")

In [ ]:
import pandas as pd

In [ ]:
# Создание таблицы сравнения
comparison_data = []
for method, result in results.items():
    row = {'method': method}
    row.update(result['metrics'])
    comparison_data.append(row)

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('f1_score', ascending=False)

print("\nСравнение методов:")
print(df_comparison.to_string())

# Сохранение в CSV
df_comparison.to_csv('../results/methods_comparison.csv', index=False)
print(f"\nТаблица сохранена: ../results/methods_comparison.csv")

# Экспорт результатов для отчета
print("\nЭкспорт результатов для отчета...")

# Создание финальной визуализации
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

In [ ]:
# Лучший метод
best_method = df_comparison.iloc[0]['method']
best_mask = results[best_method]['mask']

axes[0, 0].imshow(img1_proc, cmap='gray')
axes[0, 0].set_title('Изображение T1')
axes[0, 0].axis('off')

axes[0, 1].imshow(img2_proc, cmap='gray')
axes[0, 1].set_title('Изображение T2')
axes[0, 1].axis('off')

axes[1, 0].imshow(best_mask, cmap='gray')
axes[1, 0].set_title(f'Маска изменений ({best_method})')
axes[1, 0].axis('off')

axes[1, 1].imshow(visualizer._create_overlay(img1_proc, best_mask))
axes[1, 1].set_title('Наложение маски на T1')
axes[1, 1].axis('off')

plt.suptitle('Лучший результат обнаружения изменений', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/best_result_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nЭксперименты завершены!")